In [87]:
import math
from itertools import product
from math import floor, ceil
from copy import copy
from fractions import Fraction
import numpy as np

def extract_degree_sequence(M):
    s =M.shape[0]
    degree_sequence = []
    for i in range(s):
        degree_v_i = 0
        for j in range(s):
            if(i!=j):
                degree_v_i += M[i][j]
        degree_sequence.append(degree_v_i)
    return degree_sequence

def extract_weight_vector(M):
    s =M.shape[0]
    weight_sequence = []
    for i in range(s):
        weight_sequence.append(M[i, i])
    return weight_sequence

def char_square(M, k):
        """Given a characteristic vector k, compute some basic properties.

        Parameters
        ----------
        k: list
            A list of integers [x_1, ..., x_s] where s is the number of vertices
            of the plumbing.

        Returns
        -------
        tuple
            (a,b,c, d) where: a is a string which says if the associated spin^c
            structure on the plumbed 3-manifold is torsion or non-torsion, b is
            the order of the 1st Chern class of the associated spin^c structure
            on the plumbed 3-manifold, c is the square of the 1st Chern class of
            the associated spin^c structure on the plumbed 4-manifold (in
            other words, c = k^2), d is the t-variable normalization.

        """
        s = M.shape[0]
        M_inv = np.linalg.inv(M)
        k = np.array(k)
        k_T = np.transpose(k)
        h = np.matmul(M_inv, k)
        square = np.matmul(k_T, h)
        return square

def zhat(M, s_rep, n):
        """
        Computes the generalized :math:`\widehat{Z}` for the first n levels with
        respect to some admissible family A. If no admissible family A is
        specified then, it computes the usual zhat with respect to the
        admissible family :math:`\widehat{F}`. See :cite:p:`AJK`
        or :cite:p:`GPPV` for more details.

        Parameters
        ----------
        M: framing matrix

        s_rep: list
            A list of integers [x_1, ..., x_s] where s is the number of vertices
            in the plumbing.

        n: int
            A positive integer.

        Returns
        -------
        Laurent polynomial:
            A Laurent polynomial in q (except with the q variable
            possibly assuming powers shifted by some overall rational number).
            If :math:`\widehat{Z} = a_{1}q^{\Delta + m} + a_{2}q^{\Delta + m +1} + \cdots + a_{n}q^{\Delta + m+n-1} + \cdots`

            where m is the minimum of :math:`\chi_{k}` over the lattice, then
            the output of this function is
            :math:`a_{1}q^{\Delta + m} + a_{2}q^{\Delta + m +1} + \cdots + a_{n}q^{\Delta + m+n-1}`

            Note, some :math:`a_{i}` coefficients may be zero, in which case
            it may appear that there are less than n terms.
        """
        s = M.shape[0]
        s_rep = np.array(s_rep)
        degree_vector = extract_degree_sequence(M)
        weight_vector = extract_weight_vector(M)
        #try:
        for i in range(0, s):
            if (float(s_rep[i])-float(degree_vector[i])) % 2 != 0:
                raise Exception("Your selected spin^c convention is to"
                                " use vectors congruent mod 2 to the"
                                " degree vector, but second parameter"
                                " does not satisfy this.")
        k = s_rep + weight_vector + degree_vector
        #print(k)
        a = s_rep

        #k_squared = self.char_vector_properties(k.T)[2] 
        k_squared = char_square(M, np.transpose(k))
        #print(k_squared)
        normalization_term = -(k_squared + 3*s
                                + sum(weight_vector))/4\
                                + sum(k)/2\
                                - sum(weight_vector + degree_vector)/4
        M_inv = np.linalg.inv(M)

        #min_vector = -(1/2)*M_inv*a
        a_T = np.transpose(a)
        a_quadratic_form = np.matmul(np.matmul(a_T, M_inv), a)
        min_level = (a_quadratic_form)/4

        bounding_box = []
        for i in range(0, s):
            if min_level % 1 == 0:
                x = 2*math.sqrt(-(weight_vector[i])*(n-1))
            else:
                x = 2*math.sqrt(-(weight_vector[i])*n)
            bounding_box.append([-x, x])

        F_supp = []
        for i in range(0, s):
            if degree_vector[i] == 0:
                if bounding_box[i][0]<= -2 and bounding_box[i][1]>= 2:
                    F_supp.append([-2, 0, 2])
                elif bounding_box[i][0]<= -2 and bounding_box[i][1]< 2:
                    F_supp.append([-2, 0])
                elif bounding_box[i][0]> -2 and bounding_box[i][1]>= 2:
                    F_supp.append([0, 2])
                else:
                    F_supp.append([0])
            elif degree_vector[i] == 1:
                if bounding_box[i][0]<= -1 and bounding_box[i][1]>= 1:
                    F_supp.append([-1, 1])
                elif bounding_box[i][0]<= -1 and bounding_box[i][1]<1:
                    F_supp.append([-1])
                elif bounding_box[i][0]> -1 and bounding_box[i][1]>=1:
                    F_supp.append([1])
                else:
                    return 0
            elif degree_vector[i] == 2:
                F_supp.append([0])
            else:
                r = degree_vector[i]-2
                values = []
                if bounding_box[i][0] <=-r:
                    values.append(-r)
                    for j in range(1, floor((-r-bounding_box[i][0])/2)+1):
                        values.append(-r - 2*j)
                if bounding_box[i][1] >=r:
                    values.append(r)
                    for j in range(1, floor((bounding_box[i][1]-r)/2)+1):
                        values.append(r + 2*j)
                if len(values)==0:
                    return 0
                F_supp.append(copy(values))
        iterator = product(*F_supp)

        def F_hat_pre_comp(y):
            F = 1
            for i in range(0, s):
                if degree_vector[i] == 0:
                    if y[i] == 0:
                        F = -2*F
                    elif y[i] != 2 and y[i]!= -2:
                        F = 0
                        return F
                elif degree_vector[i] == 1:
                    if y[i] == 1:
                        F = -F
                    elif y[i] != -1:
                        F = 0
                        return F
                elif degree_vector[i] == 2:
                    if y[i] != 0:
                        F = 0
                        return F
                else:
                    if abs(y[i]) >= degree_vector[i]-2:
                        F = F*Fraction(1, 2)*(np.sign(y[i])**degree_vector[i])
                        F = F*math.comb(int(Fraction(degree_vector[i]
                                        + abs(y[i]), 2))-2, degree_vector[i]-3)
                        """F = F*(1/2)*sign(y[i])^(degree_vector[i])
                        F = F*binomial((degree_vector[i]
                                        + abs(y[i]))/2-2,
                                        degree_vector[i] -3)"""
                    else:
                        F = 0
                        return F
            return F

        exponents = [ceil(min_level) + i for i in range(0, n)]
        coefficients = np.zeros((n,))
        #coefficients = n*[0]

        for y in iterator:
            #y = Matrix(y).T
            y = np.array(y)
            #c = -((y.T*M_inv*y)[0,0])/4
            y_T = np.transpose(y)
            c = -1*(np.matmul(np.matmul(y_T, M_inv), y))/4
            #if frac(min_level) == 0:
            if (min_level %1 ==0):
                if c <= n-1:
                    #x = (1/2)*M_inv*(y-a)
                    x = Fraction(1,2)*np.matmul(M_inv, y-a)
                    #print(issubclass(x.dtype('int32').type))
                    #if x in MatrixSpace(ZZ, s, 1):
                    if np.all(np.mod(x, 1) == 0.0):
                        ind = c
                        coefficients[ind] = coefficients[ind] + F_hat_pre_comp(y)

            else:
                if c <= n:
                    #x = (1/2)*M_inv*(y-a)
                    x = Fraction(1,2)*np.matmul(M_inv, y-a)
                    #if x in MatrixSpace(ZZ, s, 1):
                    if np.all(np.mod(x, 1) == 0.0):
                        ind = floor(c)
                        coefficients[ind] = coefficients[ind] + F_hat_pre_comp(y)

        zhat_list = [(coefficients[i], exponents[i] + normalization_term) for i in range(0, n)]

        return zhat_list
        """R.<q> = PuiseuxSeriesRing(QQ)

        truncated_zhat = 0
        for x in zhat_list:
            truncated_zhat = truncated_zhat + x[0]*q^(x[1])
        return truncated_zhat"""

        """except Exception as e:
            print(e)"""
            

In [88]:
M = np.array([[-1, 1, 1, 1],
              [1, -2, 0, 0], 
              [1, 0, -3, 0],
              [1, 0, 0, -7]])


degree_sequence = extract_degree_sequence(M)
z_hat_list = zhat(M, degree_sequence, 300)
#print(z_hat_list)
for tuple in z_hat_list:
    if(tuple[0] != 0):
        print('{c}q^{e}'.format(c=tuple[0], e=tuple[1]))

1.0q^0.5
-1.0q^1.5
-1.0q^5.5
1.0q^10.5
-1.0q^11.5
1.0q^18.5
1.0q^30.5
-1.0q^41.5
1.0q^43.5
-1.0q^56.5
-1.0q^76.5
1.0q^93.5
-1.0q^96.5
1.0q^115.5
1.0q^143.5
-1.0q^166.5
1.0q^170.5
-1.0q^195.5
-1.0q^231.5
1.0q^260.5
-1.0q^265.5
1.0q^296.5
